# Rec Sys - Implicit Feedback

**Bayesian Personalized Ranking pairwise loss:**  
Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.

**Weighted Approximate-Rank Pairwise loss**:  
Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found. Useful when only positive interactions are present and optimising the top of the recommendation list (precision@k) is desired.WARP deals with (user, positive item, negative item) triplets. 


This procedure yields roughly the following algorithm:

- For a given (user, positive item pair), sample a negative item at random from all the remaining items. Compute predictions for both items; if the negative item’s prediction exceeds that of the positive item plus a margin, perform a gradient update to rank the positive item higher and the negative item lower. If there is no rank violation, continue sampling negative items until a violation is found.

- If you found a violating negative example at the first try, make a large gradient update: this indicates that a lot of negative items are ranked higher than positives items given the current state of the model, and the model must be updated by a large amount. If it took a lot of sampling to find a violating example, perform a small update: the model is likely close to the optimum and should be updated at a low rate.

In [2]:
#Get the data

import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

**Movie lens dataset:**

![](img/movielens.png)
GroupLens Research has collected and made available rating data sets from the MovieLens web site (http://movielens.org). The data sets were collected over various periods of time, depending on the size of the set.

http://grouplens.org/datasets/movielens/

**Fetch movielens 100k dataset**

The dataset contains 100,000 interactions from 1000 users on 1700 movies, and is exhaustively described
#in its README http://files.grouplens.org/datasets/movielens/ml-100k-README.txt

This data set consists of:
- 100,000 ratings (1-5) from 943 users on 1682 movies. 
- Each user has rated at least 20 movies. 
- Simple demographic info for the users (age, gender, occupation, zip)

        
        

In [3]:
movielens = fetch_movielens() 


In [ ]:
#print movie lens 
for key, value in movielens.items():
    print(key, value)

In [14]:
train = movielens['train']
test = movielens['test']

In [15]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

#### BPR Model

We’ll use two metrics of accuracy: precision@k and ROC AUC

Both are ranking metrics: to compute them, we’ll be constructing recommendation lists for all of our users, and checking the ranking of known positive movies. 

For precision at k we’ll be looking at whether they are within the first k results on the list; for AUC, we’ll be calculating the probability that any known positive is higher on the list than a random negative example.

In [16]:
model = LightFM(learning_rate=0.05, loss='bpr')

In [17]:
model.fit(train, epochs=10)

In [18]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

In [19]:
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

In [20]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))


Precision: train 0.42, test 0.06.
AUC: train 0.84, test 0.82.


#### WARP Model

In [21]:
model = LightFM(learning_rate=0.05, loss='warp')

In [22]:
model.fit_partial(train, epochs=10)

In [23]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

In [24]:
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

In [25]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.61, test 0.11.
AUC: train 0.94, test 0.90.


#### Discuss the results

1. From the model output, what do you think?
2. Which model do you think will run faster? 

### Prototype Rec Sys

In [26]:
#Let's quickly prototype a movie recommendation system

In [29]:
import numpy as np
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

In [30]:
data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


In [31]:
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)

CPU times: user 436 ms, sys: 1.83 ms, total: 438 ms
Wall time: 437 ms


In [32]:
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

Train precision: 0.39
Test precision: 0.05


Let's sample a couple of users and get their recommendations. 

To make predictions for given user, we pass the id of that user and the ids of all products we want predictions for into the predict method.

In [33]:
def sample_recommendation(model, data, user_ids):
    
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)
        

In [34]:
sample_recommendation(model, data, [3, 25, 450]) 

User 3
     Known positives:
        Contact (1997)
        Air Force One (1997)
        In & Out (1997)
     Recommended:
        Air Force One (1997)
        Assignment, The (1997)
        Postman, The (1997)
User 25
     Known positives:
        Fargo (1996)
        Godfather, The (1972)
        L.A. Confidential (1997)
     Recommended:
        Titanic (1997)
        L.A. Confidential (1997)
        Fargo (1996)
User 450
     Known positives:
        Event Horizon (1997)
        Scream (1996)
        Conspiracy Theory (1997)
     Recommended:
        Scream (1996)
        Ransom (1996)
        Conspiracy Theory (1997)


### References

- http://lyst.github.io/lightfm/docs/examples/movielens_implicit.html
- http://www.slideshare.net/maxharp3r/the-movielens-datasets-history-and-context
    